<a href="https://colab.research.google.com/github/mctavares/AI/blob/GenAISQLAgent/Copy_of_GenAISQLAgent.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Simple Gen AI SQL Agent
Ih this notebook we create a simple SQL Agent using Hugging Face and LangChain.


# References
## Data source:
https://www.kaggle.com/code/dimarudov/data-analysis-using-sql

## SQLAlchemy:
https://www.datacamp.com/tutorial/sqlalchemy-tutorial-examples
https://www.slingacademy.com/article/sqlalchemy-how-to-bulk-insert-data-into-a-table/
https://docs.sqlalchemy.org/en/20/orm/join_conditions.html
https://www.sqlitetutorial.net/sqlite-go/create-table/

## Hugging Face
https://huggingface.co/learn/cookbook/en/agent_text_to_sql

In [ ]:
# Check to see if dependencies are already installed
import transformers
import torch
import sqlalchemy
import transformers.agents

print('Hugging Face Transformers version:' + transformers.__version__)
print('PyTorch version:' + transformers.__version__)
print('sqlalchemy version:' + sqlalchemy.__version__)

Hugging Face Transformers version:4.44.2
PyTorch version:4.44.2
sqlalchemy version:2.0.36


In [10]:
# Add here list of depedencies thta need to be installed
# !pip install sqlalchemy
!pip install fsspec
'''
!pip install transformers[agents]
!pip install transformers[tools]
'''
!pip install --upgrade transformers
!pip install transformers[agents]
!pip install --upgrade sqlalchemy

In [44]:
# Let's create a simple database
import sqlalchemy as db

# Based on https://www.kaggle.com/code/alaasedeeq/european-soccer-database-with-sqlite3
# engine = db.create_engine("sqlite:///european_database.sqlite")

metadata = db.MetaData()

# Country table
country = db.Table('country', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Name', db.String(255), nullable=False)
              )
# League table
league = db.Table('league', metadata,
              db.Column('Id', db.Integer(),primary_key=True),
              db.Column('Country_Id', db.Integer(),db.ForeignKey('country.Id')),
              db.Column('Name', db.String(255), nullable=False)
              )
engine = db.create_engine('sqlite:///:memory:') #, echo=True) # Comment out for verbose creation
metadata.create_all(engine)
conn = engine.connect()
"""
inspector = db.inspect(engine)

for table_name in inspector.get_table_names():
  print("Table: %s" % table_name)
  for column in inspector.get_columns(table_name):
    print("Column: %s" % column['name'])

print('Done.')
###
# interact with database
#print("Database opened.")
#conn = engine.connect()
#metadata = db.MetaData()
#matches= db.Table('Match', metadata, autoload_with=engine) #Table object

#print(repr(metadata.tables['Match']))
#print (repr(metadata))
"""

'\ninspector = db.inspect(engine)\n\nfor table_name in inspector.get_table_names():\n  print("Table: %s" % table_name)\n  for column in inspector.get_columns(table_name):\n    print("Column: %s" % column[\'name\'])\n\nprint(\'Done.\')\n###\n# interact with database\n#print("Database opened.")\n#conn = engine.connect()\n#metadata = db.MetaData()\n#matches= db.Table(\'Match\', metadata, autoload_with=engine) #Table object\n\n#print(repr(metadata.tables[\'Match\']))\n#print (repr(metadata))\n'

In [45]:
insert_stmt = country.insert().values([
{'Id':1, 'Name':'Belgium'},
{'Id':1729, 'Name':'England'},
{'Id':4769, 'Name':'France'},
{'Id':7809, 'Name':'Germany'},
{'Id':10257, 'Name':'Italy'},
{'Id':13274, 'Name':'Netherlands'},
{'Id':15722, 'Name':'Poland'},
{'Id':17642, 'Name':'Portugal'},
{'Id':19694, 'Name':'Scotland'},
{'Id':21518, 'Name':'Spain'},
{'Id':24558, 'Name':'Switzerland'}
])

#conn = engine.connect()
conn.execute(insert_stmt)
output = conn.execute(country.select()).fetchall()
print(output)
#conn.close()

print ('Done')

[(1, 'Belgium'), (1729, 'England'), (4769, 'France'), (7809, 'Germany'), (10257, 'Italy'), (13274, 'Netherlands'), (15722, 'Poland'), (17642, 'Portugal'), (19694, 'Scotland'), (21518, 'Spain'), (24558, 'Switzerland')]
Done


In [46]:
insert_stmt = league.insert().values([
{'Id':1,'Country_Id':1,'Name':'Belgium Jupiler League'},
{'Id':1729,'Country_Id':1729,'Name':'England Premier League'},
{'Id':4769,'Country_Id':4769,'Name':'France Ligue 1'},
{'Id':7809,'Country_Id':7809,'Name':'Germany 1. Bundesliga'},
{'Id':10257,'Country_Id':10257,'Name':'Italy Serie A'},
{'Id':13274,'Country_Id':13274,'Name':'Netherlands Eredivisie'},
{'Id':15722,'Country_Id':15722,'Name':'Poland Ekstraklasa'},
{'Id':17642,'Country_Id':17642,'Name':'Portugal Liga ZON Sagres'},
{'Id':19694,'Country_Id':19694,'Name':'Scotland Premier League'},
{'Id':21518,'Country_Id':21518,'Name':'Spain LIGA BBVA	21518'},
{'Id':24558,'Country_Id':24558,'Name':'Switzerland Super League'}
])
#conn = engine.connect()
conn.execute(insert_stmt)
output = conn.execute(league.select()).fetchall()
#conn.close()
print(output)

[(1, 1, 'Belgium Jupiler League'), (1729, 1729, 'England Premier League'), (4769, 4769, 'France Ligue 1'), (7809, 7809, 'Germany 1. Bundesliga'), (10257, 10257, 'Italy Serie A'), (13274, 13274, 'Netherlands Eredivisie'), (15722, 15722, 'Poland Ekstraklasa'), (17642, 17642, 'Portugal Liga ZON Sagres'), (19694, 19694, 'Scotland Premier League'), (21518, 21518, 'Spain LIGA BBVA\t21518'), (24558, 24558, 'Switzerland Super League')]


In [47]:
# Hugging Face code
from transformers import ReactCodeAgent, HfApiEngine, Tool, tool

'''
class QueryTool(Tool):
  def __init__(self):
    super().__init__(name="query_tool", description="A tool that generates a brief summary of input text.")

  def __call__(self, query: str) -> str:
    output = ""
    with engine.connect() as con:
      rows = con.execute(db.text(query))
      for row in rows:
        output += "\n" + str(row)
    return output

query_tool = QueryTool()
'''
@tool
def query_tool(query: str) -> str:
  """
  Allows you to perform SQL queries on the table. Returns a string representation of the result.
  The table is named 'country'. Its description is as follows:
  Columns:
  - Id: INTEGER
  - Name: VARCHAR(16)

  Args:
  query: The query to perform. This should be correct SQL.
  """
  output = ""
  rows = conn.execute(db.text(query))
  for row in rows:
    print('here')
    output += "\n" + str(row)
  return output





agent = ReactCodeAgent(
    tools=[query_tool],
    llm_engine=HfApiEngine("meta-llama/Meta-Llama-3-8B-Instruct"),
)

agent.run("Can you give me the name of a country that starts with the letter p?")

conn.close()
print ('Done')

======== New task ========
Can you give me the name of a country that starts with the letter p?
=== Agent thoughts:
Thought: I will use the `query_tool` to perform a SQL query on the 'country' table to find a country that starts with the letter 'P'.
>>> Agent is executing the code below:
countries = query_tool(query="SELECT Name FROM country WHERE Name LIKE 'P%'")
for country in countries.splitlines():
    print(country)
====
Print outputs:

('Poland',)
('Portugal',)



here
here


=== Agent thoughts:
Thought: Now that I have the names of countries that start with the letter 'P', I need to use the `final_answer` tool to provide the final answer.
>>> Agent is executing the code below:
final_answer("Poland or Portugal")
====
Print outputs:

Last output from code snippet:
Poland or Portugal
Final answer:
Poland or Portugal


Done


In [ ]:
from transformers import GPT2LMHeadModel, GPT2Tokenizer

# Remove warning: FutureWarning: `clean_up_tokenization_spaces` was not set.
tokenizer = GPT2Tokenizer.from_pretrained("gpt2", clean_up_tokenization_spaces=False)
model = GPT2LMHeadModel.from_pretrained("gpt2")

input_text = "Generate a SQL query to get all orders from the last month."
input_ids = tokenizer.encode(input_text, return_tensors='pt')
# Removed warning: The attention mask and the pad token id were not set. As a consequence, you may observe unexpected behavior.
output = model.generate(input_ids, max_length=100, pad_token_id=tokenizer.eos_token_id)

print(tokenizer.decode(output[0], skip_special_tokens=True))


Generate a SQL query to get all orders from the last month.

The query will be executed in the order column.

The order column will contain the order number.

The order column will contain the order number.

The order column will contain the order number.

The order column will contain the order number.

The order column will contain the order number.

The order column will contain the order number.

The order column will contain the
